In [1]:
import pandas as pd
import re

In [2]:
# Cargar CSVs
grados = pd.read_csv("../Output/grados.csv", sep=";")
grados_ofertados = pd.read_csv("../Output/grados_ofertados.csv", sep=";")
facultades = pd.read_csv("../Output/facultad.csv", sep=";")
universidades = pd.read_csv("../Output/universidad.csv", sep=";")
notas_corte = pd.read_csv("../Input/notas_corte.csv", sep=",")

In [3]:
ofertas = (
    grados_ofertados
    .merge(grados, left_on="id_grado", right_on="id", how="left", suffixes=("_of", "_gr"))
    .merge(facultades, left_on="id_facultad", right_on="id", how="left", suffixes=("", "_fac"))
    .merge(universidades, left_on="id_universidad", right_on="id", how="left", suffixes=("", "_uni"))
)

ofertas_simpl = ofertas[["id_of", "nombre", "nombre_uni"]].rename(
    columns={"nombre": "grado", "nombre_uni": "universidad"}
)

ofertas_simpl.to_csv("../Output/prueba.csv", sep=";", index=False)


In [ ]:
merged = notas_corte.merge(
    ofertas_simpl,
    on=["universidad", "grado"],
    how="left"
)

nota_corte = merged[["id_of", "año", "convocatoria", "grupo", "nota"]].rename(
    columns={"id_of": "id_grado_ofertado"}
)

nota_corte["nota"] = nota_corte["nota"].astype(float)

nota_corte = (
    nota_corte
    .sort_values("nota", ascending=False)
    .drop_duplicates(subset=["id_grado_ofertado", "año", "convocatoria", "grupo"], keep="first")
    .reset_index(drop=True)
)

nota_corte.insert(0, "id", range(1, len(nota_corte) + 1))
nota_corte = nota_corte.dropna(subset=["id_grado_ofertado"])
nota_corte["id_grado_ofertado"] = nota_corte["id_grado_ofertado"].astype(int)

nota_corte.to_csv("../Output/nota_corte.csv", sep=";", index=False)